## 1. Einleitung / Ziel des Projekts 🏠

In diesem Projekt untersuchen wir die Verkaufspreise von Häusern im Raum Seattle (King County, USA).  
Wir wollen die wichtigsten Preisfaktoren identifizieren, wie z. B.:
- **Wohnfläche, Zimmeranzahl, Baujahr, Zustand**
- **Einfluss der Lage (Wasserlage, Postleitzahl)**
- **Zusammenhänge zwischen Preis und Wohnqualität**

Ziel ist es, **Auffälligkeiten und Muster** im Immobilienmarkt zu erkennen.


## 2. Daten laden und Überblick 🧐

Wir starten mit dem Laden der Daten und einem ersten Überblick:
- **Wie viele Zeilen und Spalten gibt es?**
- **Welche Spalten sind enthalten?**
- **Gibt es fehlende Werte?**


In [1]:
# Bibliotheken importieren
import pandas as pd

# Daten laden
df = pd.read_csv("../data/kc_house_data.csv")

# Ersten Blick auf die Daten werfen
df.head()


,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,20141013T000000,221900.0,3,1.00,1180,5650,1.0,0,0,...,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,6414100192,20141209T000000,538000.0,3,2.25,2570,7242,2.0,0,0,...,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639
2,5631500400,20150225T000000,180000.0,2,1.00,770,10000,1.0,0,0,...,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
3,2487200875,20141209T000000,604000.0,4,3.00,1960,5000,1.0,0,0,...,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000
4,1954400510,20150218T000000,510000.0,3,2.00,1680,8080,1.0,0,0,...,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503


## 3. Erste Datenprüfung 🧪

Ziel:
- **Größe des Datensatzes ermitteln**
- **Spaltennamen und Datentypen prüfen**
- **Fehlende Werte identifizieren**


In [2]:
# Größe des Datensatzes
df.shape

# Spaltennamen anzeigen
df.columns

# Datentypen und Speicherverbrauch prüfen
df.info()

# Fehlende Werte prüfen
df.isnull().sum()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21613 entries, 0 to 21612
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             21613 non-null  int64  
 1   date           21613 non-null  object 
 2   price          21613 non-null  float64
 3   bedrooms       21613 non-null  int64  
 4   bathrooms      21613 non-null  float64
 5   sqft_living    21613 non-null  int64  
 6   sqft_lot       21613 non-null  int64  
 7   floors         21613 non-null  float64
 8   waterfront     21613 non-null  int64  
 9   view           21613 non-null  int64  
 10  condition      21613 non-null  int64  
 11  grade          21613 non-null  int64  
 12  sqft_above     21613 non-null  int64  
 13  sqft_basement  21613 non-null  int64  
 14  yr_built       21613 non-null  int64  
 15  yr_renovated   21613 non-null  int64  
 16  zipcode        21613 non-null  int64  
 17  lat            21613 non-null  float64
 18  long  

id               0
date             0
price            0
bedrooms         0
bathrooms        0
sqft_living      0
sqft_lot         0
floors           0
waterfront       0
view             0
condition        0
grade            0
sqft_above       0
sqft_basement    0
yr_built         0
yr_renovated     0
zipcode          0
lat              0
long             0
sqft_living15    0
sqft_lot15       0
dtype: int64

## 4. Daten bereinigen 🧹

Die Daten sind insgesamt **sauber** – keine fehlenden Werte.  
Einzig die `date`-Spalte liegt noch als `object` (Text) vor und wird jetzt in ein echtes **Datumsformat (datetime)** umgewandelt.


In [3]:
# Datumsspalte umwandeln
df['date'] = pd.to_datetime(df['date'])

# Prüfen, ob es geklappt hat
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21613 entries, 0 to 21612
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   id             21613 non-null  int64         
 1   date           21613 non-null  datetime64[ns]
 2   price          21613 non-null  float64       
 3   bedrooms       21613 non-null  int64         
 4   bathrooms      21613 non-null  float64       
 5   sqft_living    21613 non-null  int64         
 6   sqft_lot       21613 non-null  int64         
 7   floors         21613 non-null  float64       
 8   waterfront     21613 non-null  int64         
 9   view           21613 non-null  int64         
 10  condition      21613 non-null  int64         
 11  grade          21613 non-null  int64         
 12  sqft_above     21613 non-null  int64         
 13  sqft_basement  21613 non-null  int64         
 14  yr_built       21613 non-null  int64         
 15  yr_renovated   2161

## Schritt 5: Explorative Datenanalyse (EDA)

Jetzt gehen wir die ersten spannenden Fragen an:

- ** Wie verteilen sich die Immobilienpreise? **
- ** Gibt es viele günstige oder eher teure Häuser? **
- ** Wie verteilen sich die Wohnflächen? **
- ** Sind die meisten Häuser klein oder groß? **
- ** Hat Wasserlage (waterfront) Einfluss auf den Preis? **
- ** Sind Häuser am Wasser teurer? **
- ** Welche Postleitzahlen haben die höchsten Preise? **	
- ** Gibt es teure Gegenden / Hotspots? **
- ** Wie entwickeln sich Preise über die Zeit? **	
- ** Steigen die Preise, oder gibt es Schwankungen? **

## 5.1 Verteilung der Immobilienpreise 💰📊

Wie verteilen sich die Hauspreise?  
Gibt es viele günstige Immobilien oder ziehen einige Luxusobjekte den Schnitt hoch?


In [8]:
plt.figure(figsize=(10, 5))
sns.histplot(df['price'], bins=50, kde=True)

plt.title("Verteilung der Immobilienpreise")
plt.xlabel("Preis (€)")
plt.ylabel("Anzahl der Immobilien")

# Speichern
plt.savefig("../visuals/preisverteilung.png")
plt.close()  # Schließt das Plot-Fenster (damit es nicht doppelt angezeigt wird)


## 5.1 Verteilung der Immobilienpreise – Kennzahlen 🧮

Neben der Visualisierung werfen wir einen Blick auf die wichtigsten Kennzahlen:
- **Minimum, Maximum, Median**
- **Mittelwert, Standardabweichung**
- **Quartile**


In [12]:
# Statistische Kennzahlen für Preis
preis_statistik = df['price'].describe()
preis_statistik


count    2.161300e+04
mean     5.400881e+05
std      3.671272e+05
min      7.500000e+04
25%      3.219500e+05
50%      4.500000e+05
75%      6.450000e+05
max      7.700000e+06
Name: price, dtype: float64

## 5.2 Wohnfläche (sqft_living) – Verteilung 🏠📏

Wie groß sind die meisten Häuser?  
Gibt es viele kleine Häuser oder überwiegen große Immobilien?


In [9]:
plt.figure(figsize=(10, 5))
sns.histplot(df['sqft_living'], bins=50, kde=True)

plt.title("Verteilung der Wohnfläche")
plt.xlabel("Wohnfläche (sqft)")
plt.ylabel("Anzahl der Immobilien")

# Speichern
plt.savefig("../visuals/wohnflaeche_verteilung.png")
plt.close()


## 5.2 Wohnfläche (sqft_living) – Kennzahlen 🏠📏

Wie verteilen sich die Wohnflächen in Zahlen?


In [13]:
# Statistische Kennzahlen für Wohnfläche
wohnflaeche_statistik = df['sqft_living'].describe()
wohnflaeche_statistik


count    21613.000000
mean      2079.899736
std        918.440897
min        290.000000
25%       1427.000000
50%       1910.000000
75%       2550.000000
max      13540.000000
Name: sqft_living, dtype: float64

## 5.3 Wasserlage (waterfront) vs. Preis 🌊💰

Beeinflusst die Nähe zum Wasser den Immobilienpreis?  
Häuser mit Wasserlage (`waterfront = 1`) könnten teurer sein als Häuser ohne (`waterfront = 0`).


In [10]:
# Durchschnittspreis mit und ohne Wasserlage
waterfront_prices = df.groupby('waterfront')['price'].mean()

# Ausgabe der Durchschnittspreise
waterfront_prices


waterfront
0    5.315636e+05
1    1.661876e+06
Name: price, dtype: float64

In [11]:
plt.figure(figsize=(6, 4))
sns.barplot(x=waterfront_prices.index, y=waterfront_prices.values, palette='Blues')

plt.title("Durchschnittlicher Preis: Wasserlage vs. keine Wasserlage")
plt.xlabel("Wasserlage (0 = Nein, 1 = Ja)")
plt.ylabel("Durchschnittlicher Preis (€)")

# Speichern
plt.savefig("../visuals/wasserlage_preisvergleich.png")
plt.close()


/var/folders/yy/b7kbx5nj53l4y_x6xkz84z2r0000gn/T/ipykernel_43176/1674665245.py:2: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.barplot(x=waterfront_prices.index, y=waterfront_prices.values, palette='Blues')


## 5.3 Wasserlage – Durchschnittspreise 💧📊

Wie unterscheiden sich die **durchschnittlichen Preise** zwischen Häusern mit und ohne Wasserlage?


In [14]:
waterfront_prices


waterfront
0    5.315636e+05
1    1.661876e+06
Name: price, dtype: float64

## 5.4 Zusammenfassung – Kennzahlen für Preis, Wohnfläche und Wasserlage 🧮

Die wichtigsten statistischen Kennzahlen auf einen Blick:
- **Preis:** Minimum, Maximum, Median, Quartile, Mittelwert
- **Wohnfläche:** Verteilung der Quadratfuß
- **Wasserlage:** Durchschnittspreise für Häuser mit/ohne Wasserlage


In [15]:
# Preise – Verteilung
print("📊 Immobilienpreise – Kennzahlen")
print(df['price'].describe())
print("\n")

# Wohnfläche – Verteilung
print("🏠 Wohnfläche (sqft_living) – Kennzahlen")
print(df['sqft_living'].describe())
print("\n")

# Wasserlage – Durchschnittspreise
print("💧 Wasserlage vs. Preis – Durchschnittspreise")
print(df.groupby('waterfront')['price'].mean())


📊 Immobilienpreise – Kennzahlen
count    2.161300e+04
mean     5.400881e+05
std      3.671272e+05
min      7.500000e+04
25%      3.219500e+05
50%      4.500000e+05
75%      6.450000e+05
max      7.700000e+06
Name: price, dtype: float64


🏠 Wohnfläche (sqft_living) – Kennzahlen
count    21613.000000
mean      2079.899736
std        918.440897
min        290.000000
25%       1427.000000
50%       1910.000000
75%       2550.000000
max      13540.000000
Name: sqft_living, dtype: float64


💧 Wasserlage vs. Preis – Durchschnittspreise
waterfront
0    5.315636e+05
1    1.661876e+06
Name: price, dtype: float64


## 5.5 Interpretation der Kennzahlen 🧠

### 1️⃣ Immobilienpreise:
- **Typischer Preis: ~450.000 €** (Median), **Durchschnitt: ~540.000 €**.
- **Luxusimmobilien (bis 7,7 Mio €) ziehen den Durchschnitt hoch.**
- **50% der Häuser kosten zwischen 322.000 € und 645.000 €.**

### 2️⃣ Wohnfläche:
- **Typische Wohnfläche: ~178 m²** (Median), **Durchschnitt: ~193 m²**.
- **Hohe Streuung → Tiny-Häuser (~27 m²) bis Villen (1.250 m²).**
- **50% der Häuser liegen zwischen ~132 m² und 237 m².**

### 3️⃣ Wasserlage:
- **Ohne Wasserlage: ~531.000 €** im Durchschnitt.
- **Mit Wasserlage: ~1,66 Mio €** → **Wasserlage verdreifacht den Preis!**


## 5.6 Teure Gegenden – Postleitzahlen 🏘️📍

Wie unterscheiden sich die Preise zwischen den Postleitzahlen?  
Wir wollen wissen:
- **Welche PLZ haben die höchsten Durchschnittspreise?**
- **Gibt es bestimmte "Hotspots" mit besonders teuren Immobilien?**


In [16]:
# Durchschnittspreis pro Postleitzahl berechnen
plz_prices = df.groupby('zipcode')['price'].mean().sort_values(ascending=False).head(10)

# Ausgabe der Top 10 PLZ
plz_prices


zipcode
98039    2.160607e+06
98004    1.355927e+06
98040    1.194230e+06
98112    1.095499e+06
98102    9.012583e+05
98109    8.796236e+05
98105    8.628252e+05
98006    8.596848e+05
98119    8.494480e+05
98005    8.101649e+05
Name: price, dtype: float64

In [17]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 5))
plz_prices.plot(kind='bar', color='purple')

plt.title("Top 10 Postleitzahlen nach Durchschnittspreis")
plt.xlabel("Postleitzahl")
plt.ylabel("Durchschnittspreis (€)")

# Speichern in visuals/
plt.savefig("../visuals/top10_plz_preise.png")
plt.close()


## 5.7 Teure Gegenden – Interpretation 🧠

### Top 10 Postleitzahlen nach Durchschnittspreis:
| PLZ   | Durchschnittspreis | Region / Besonderheit |
|-------|---------------------|-----------------------|
| **98039** | 2.160.607 €       | Medina – Luxus, Bill Gates |
| **98004** | 1.355.927 €       | Bellevue – Tech-Elite |
| **98040** | 1.194.230 €       | Mercer Island – Insel |
| **98112** | 1.095.499 €       | Capitol Hill – Zentral, gehoben |
| **98102** | 901.258 €         | Eastlake – Wasserlage |
| **98109** | 879.624 €         | Queen Anne – Aussicht |
| **98105** | 862.825 €         | Uni District – Altbauten |
| **98006** | 859.684 €         | Bellevue Süd – Familien |
| **98119** | 849.448 €         | Queen Anne Nord – Skyline |
| **98005** | 810.165 €         | Bellevue Ost – Nähe Tech |

### Erkenntnisse:
- **Luxusgebiete wie Medina & Mercer Island führen die Liste an**.
- **Tech-Standorte (Bellevue, Microsoft-Nähe) treiben Preise hoch**.
- **Zentrale Lagen mit Wasser-/Skyline-Blick (Queen Anne, Eastlake) sind sehr gefragt**.
